Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
Если товар не встречался в обучающей выборке, его популярность равна 0.
Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.
Задание

На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
Реализуйте два алгоритма рекомендаций:
сортировка просмотренных id по популярности (частота появления в просмотренных),
сортировка просмотренных id по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Дополнительные вопросы

Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [44]:
import pandas as pd
import numpy as np
import math

In [2]:
train = pd.read_csv('coursera_sessions_train.txt',';',header=-1,names=["id_seen","id_buy"])
test = pd.read_csv('coursera_sessions_test.txt',';',header=-1,names=["id_seen","id_buy"])

In [94]:
train.head(10)

,id_seen,id_buy
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
5,42,NaN
6,"47,48,49",NaN
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
8,"71,72,73,74",NaN
9,"76,77,78",NaN


In [191]:
#train[7.isin(train.id_buy)]
#train.id_buy.str.contains(r'^7')==True
#pd.isnull(train.loc[0,"id_buy"])
train1 = train.dropna()
test1 = test.dropna()
train1.head(10)

,id_seen,id_buy
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86
19,"138,198,199,127",199
30,"303,304,305,306,307,308,309,310,311,312",303
33,"352,353,352",352
55,519,519
64,"599,600,601,602","603,604,602,599,605,606,600"
72,"687,688,689,690,691,690,688,690,688,692","690,688"
89,"850,851,852",851
93,"879,884,170,137,170,879,884,879,885,886,879,88...",879


In [204]:
train1.iloc[0]

id_seen    59,60,61,62,60,63,64,65,66,61,67,68,67
id_buy                                   67,60,63
Name: 7, dtype: object

In [299]:
catalog = dict()
for i in range(train.shape[0]):
    for j in train.loc[i,"id_seen"].split(','):
        if j in catalog:
            catalog[j][0] += 1
        else:
            catalog[j] = [1,0,i]
    if not pd.isnull(train.loc[i,"id_buy"]):
        for j in train.loc[i,"id_buy"].split(','):
            if j in catalog:
                catalog[j][1] += 1
for i in range(test.shape[0]):
    for j in test.loc[i,"id_seen"].split(','):
        if j not in catalog:
            catalog[j] = [1,0,i]
catalog
# id: seen_count, bought_count, first_session_number

{'91139': [1, 0, 39062],
 '89370': [1, 0, 37200],
 '89371': [1, 0, 37200],
 '89372': [1, 0, 37202],
 '89373': [1, 0, 37203],
 '89374': [1, 0, 37206],
 '89375': [1, 0, 37206],
 '89376': [2, 0, 37206],
 '89377': [1, 0, 37206],
 '89378': [1, 0, 37206],
 '89379': [2, 0, 37206],
 '5988': [17, 0, 12105],
 '5989': [1, 0, 737],
 '5982': [4, 0, 736],
 '5983': [1, 0, 736],
 '5980': [6, 0, 736],
 '5981': [4, 0, 736],
 '5986': [10, 0, 737],
 '5987': [9, 0, 737],
 '5984': [3, 0, 7257],
 '5985': [7, 1, 737],
 '82449': [1, 0, 31129],
 '82448': [1, 0, 31129],
 '82443': [1, 0, 31126],
 '82442': [1, 0, 31126],
 '82441': [2, 0, 31126],
 '82440': [3, 0, 31126],
 '82447': [2, 0, 31129],
 '82446': [3, 0, 31129],
 '82445': [1, 0, 31129],
 '82444': [2, 0, 31127],
 '73624': [1, 0, 24740],
 '54545': [5, 0, 13513],
 '97159': [1, 0, 45024],
 '79234': [1, 0, 28909],
 '12995': [4, 0, 28453],
 '43264': [1, 0, 9263],
 '97152': [1, 0, 45024],
 '97151': [1, 0, 45024],
 '97150': [1, 0, 45024],
 '97157': [1, 0, 45024],
 

На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
Реализуйте два алгоритма рекомендаций: сортировка просмотренных id по популярности (частота появления в просмотренных), сортировка просмотренных id по покупаемости (частота появления в покупках).
Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [300]:
sort_seen1 = sorted(catalog.items(), key=lambda x: (-x[1][0],x[1][2]))
sort_buy1 = sorted(catalog.items(), key=lambda x: (-x[1][1],x[1][2]))
sort_buy1
# id: seen_count, bought_count, first_session_number

[('158', [641, 14, 140]),
 ('204', [396, 12, 65]),
 ('73', [677, 11, 8]),
 ('3324', [204, 11, 576]),
 ('977', [64, 10, 162]),
 ('3149', [213, 10, 512]),
 ('5569', [102, 10, 684]),
 ('1181', [95, 9, 130]),
 ('162', [318, 8, 59]),
 ('1852', [172, 7, 1447]),
 ('4335', [72, 7, 2736]),
 ('3445', [58, 6, 425]),
 ('4333', [64, 6, 525]),
 ('5570', [100, 6, 684]),
 ('31007', [22, 6, 7389]),
 ('170', [280, 5, 93]),
 ('1843', [122, 5, 318]),
 ('2861', [83, 5, 347]),
 ('2929', [75, 5, 357]),
 ('4062', [82, 5, 488]),
 ('4089', [19, 5, 492]),
 ('1814', [138, 5, 636]),
 ('5501', [160, 5, 714]),
 ('7631', [29, 5, 953]),
 ('605', [34, 5, 957]),
 ('8136', [35, 5, 1006]),
 ('3793', [32, 5, 1107]),
 ('1058', [59, 5, 1179]),
 ('10886', [17, 5, 1417]),
 ('6340', [41, 5, 1517]),
 ('422', [60, 5, 3486]),
 ('11005', [30, 5, 6008]),
 ('15041', [17, 5, 10665]),
 ('71', [141, 4, 8]),
 ('262', [387, 4, 26]),
 ('519', [44, 4, 55]),
 ('743', [62, 4, 78]),
 ('751', [197, 4, 79]),
 ('135', [44, 4, 101]),
 ('1314', [50

In [301]:
sort_seen = [x[0] for x in sort_seen1]
sort_buy = [x[0] for x in sort_buy1]

In [272]:
sort_buy

['158',
 '204',
 '73',
 '3324',
 '977',
 '3149',
 '5569',
 '1181',
 '162',
 '1852',
 '4335',
 '3445',
 '4333',
 '5570',
 '31007',
 '170',
 '1843',
 '2861',
 '2929',
 '4062',
 '4089',
 '1814',
 '5501',
 '7631',
 '605',
 '8136',
 '3793',
 '1058',
 '10886',
 '6340',
 '422',
 '11005',
 '15041',
 '71',
 '262',
 '519',
 '743',
 '751',
 '135',
 '1314',
 '1319',
 '2290',
 '3650',
 '3410',
 '4815',
 '5202',
 '5390',
 '6020',
 '5894',
 '8338',
 '2070',
 '7025',
 '7686',
 '9623',
 '8162',
 '11610',
 '11899',
 '7379',
 '6202',
 '17245',
 '160',
 '16194',
 '4888',
 '31719',
 '33142',
 '20783',
 '137',
 '8',
 '28',
 '1079',
 '1118',
 '1317',
 '1315',
 '1474',
 '1651',
 '2735',
 '2964',
 '1595',
 '1869',
 '3286',
 '3443',
 '3613',
 '3612',
 '3784',
 '758',
 '1644',
 '4394',
 '4526',
 '4751',
 '1714',
 '5780',
 '1782',
 '6303',
 '6805',
 '4600',
 '6974',
 '7003',
 '7908',
 '7983',
 '7986',
 '1715',
 '9135',
 '9398',
 '2193',
 '1728',
 '9643',
 '9773',
 '10244',
 '11101',
 '11420',
 '11462',
 '1949',
 

Важно:

1. Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
2. Если товар не встречался в обучающей выборке, его популярность равна 0.
3. Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
4. Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Гарантируется, что среди id купленных товаров все различные.

AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5

In [302]:
#sort_buy[:5]
sort_buy2 = {x[0]:x[1][1] for x in sort_buy1}
#sorted(catalog.items(), key=lambda x: (-x[1][1],x[1][2]))
'158' in sort_buy2

True

In [294]:
# Train
#каждую строку: список просмотренных товаров сортировать в порядке рекомендаций и затем выбирать топ1 или топ5
seen_recall_1 = []
seen_recall_5 = []
seen_precision_1 = []
seen_precision_5 = []
buy_recall_1 = []
buy_recall_5 = []
buy_precision_1 = []
buy_precision_5 = []
for i in range(train.shape[0]):
    if not pd.isnull(train.loc[i,"id_buy"]):
        sorted_seen_row = dict()
        sorted_seen_row_by_buy = dict()
        for j in train.loc[i,"id_seen"].split(','):
            if j in sort_seen:
                sorted_seen_row[j] = sort_seen.index(j)
            if j in sort_buy2:
                sorted_seen_row_by_buy[j] = [sort_buy2[j],train.loc[i,"id_seen"].split(',').index(j)]
        sorted_seen = [x[0] for x in sorted(sorted_seen_row.items(), key=lambda x: (x[1]))]
        sorted_seen_by_buy = [x[0] for x in sorted(sorted_seen_row_by_buy.items(), key=lambda x: (-x[1][0],x[1][1]))]

        seen_recall_1.append(float(1 if sorted_seen[0] in train.loc[i,"id_buy"].split(',') else 0)/len(train.loc[i,"id_buy"].split(',')))
        seen_precision_1.append(float(1 if sorted_seen[0] in train.loc[i,"id_buy"].split(',') else 0)/1)
        buy_recall_1.append(float(1 if sorted_seen_by_buy[0] in train.loc[i,"id_buy"].split(',') else 0)/len(train.loc[i,"id_buy"].split(',')))
        buy_precision_1.append(float(1 if sorted_seen_by_buy[0] in train.loc[i,"id_buy"].split(',') else 0)/1)
        #if len(set(train.loc[i,"id_seen"].split(',')))>=5:
        seen_recall_5.append(float(len(set(train.loc[i,"id_buy"].split(',')).intersection(sorted_seen[:5])))/len(train.loc[i,"id_buy"].split(',')))
        seen_precision_5.append(float(len(set(train.loc[i,"id_buy"].split(',')).intersection(sorted_seen[:5])))/5)
        buy_recall_5.append(float(len(set(train.loc[i,"id_buy"].split(',')).intersection(sorted_seen_by_buy[:5])))/len(train.loc[i,"id_buy"].split(',')))
        buy_precision_5.append(float(len(set(train.loc[i,"id_buy"].split(',')).intersection(sorted_seen_by_buy[:5])))/5)
print(np.mean(seen_recall_1))
print(np.mean(seen_precision_1))
print(np.mean(seen_recall_5))
print(np.mean(seen_precision_5))
print(np.mean(buy_recall_1))
print(np.mean(buy_precision_1))
print(np.mean(buy_recall_5))
print(np.mean(buy_precision_5))

0.435465063084
0.503880266075
0.822569996676
0.212139689579
0.689558139877
0.80487804878
0.926501315727
0.252716186253


In [303]:
test

,id_seen,id_buy
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN
5,"43,44,43,45,43,45,43,46",NaN
6,"50,51,47,52,49,53,54,55,56,57,58",NaN
7,"63,68,69,70,66,61,59,61,66,68","66,63"
8,75,NaN
9,"79,80,81,82,83",NaN


In [305]:
# Test
#каждую строку: список просмотренных товаров сортировать в порядке рекомендаций и затем выбирать топ1 или топ5
seen_recall_1 = []
seen_recall_5 = []
seen_precision_1 = []
seen_precision_5 = []
buy_recall_1 = []
buy_recall_5 = []
buy_precision_1 = []
buy_precision_5 = []
for i in range(test.shape[0]):
    if not pd.isnull(test.loc[i,"id_buy"]):
        sorted_seen_row = dict()
        sorted_seen_row_by_buy = dict()
        for j in test.loc[i,"id_seen"].split(','):
            if j in sort_seen:
                sorted_seen_row[j] = sort_seen.index(j)
            if j in sort_buy2:
                sorted_seen_row_by_buy[j] = [sort_buy2[j],test.loc[i,"id_seen"].split(',').index(j)]
        sorted_seen = [x[0] for x in sorted(sorted_seen_row.items(), key=lambda x: (x[1]))]
        sorted_seen_by_buy = [x[0] for x in sorted(sorted_seen_row_by_buy.items(), key=lambda x: (-x[1][0],x[1][1]))]

        seen_recall_1.append(float(1 if sorted_seen[0] in test.loc[i,"id_buy"].split(',') else 0)/len(test.loc[i,"id_buy"].split(',')))
        seen_precision_1.append(float(1 if sorted_seen[0] in test.loc[i,"id_buy"].split(',') else 0)/1)
        buy_recall_1.append(float(1 if sorted_seen_by_buy[0] in test.loc[i,"id_buy"].split(',') else 0)/len(test.loc[i,"id_buy"].split(',')))
        buy_precision_1.append(float(1 if sorted_seen_by_buy[0] in test.loc[i,"id_buy"].split(',') else 0)/1)
        seen_recall_5.append(float(len(set(test.loc[i,"id_buy"].split(',')).intersection(sorted_seen[:5])))/len(test.loc[i,"id_buy"].split(',')))
        seen_precision_5.append(float(len(set(test.loc[i,"id_buy"].split(',')).intersection(sorted_seen[:5])))/5)
        buy_recall_5.append(float(len(set(test.loc[i,"id_buy"].split(',')).intersection(sorted_seen_by_buy[:5])))/len(test.loc[i,"id_buy"].split(',')))
        buy_precision_5.append(float(len(set(test.loc[i,"id_buy"].split(',')).intersection(sorted_seen_by_buy[:5])))/5)
print(np.mean(seen_recall_1))
print(np.mean(seen_precision_1))
print(np.mean(seen_recall_5))
print(np.mean(seen_precision_5))
print(np.mean(buy_recall_1))
print(np.mean(buy_precision_1))
print(np.mean(buy_recall_5))
print(np.mean(buy_precision_5))

0.413646570952
0.47721691678
0.799971050697
0.204365620737
0.460151771941
0.527421555252
0.820141958533
0.210095497954
